# Task 2: Car Recommendation
### Team McFlurry 


In this task, we use the data in "train.csv" and build a three-step model which can be extended to apply to datasets with the same structure. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import warnings
warnings.filterwarnings('ignore')

To illustrate our model in detail, we assume `listing_id` (the unique id for the listing of a used car) equals  **1029310** and `k` (the number of recommendations) equals 10 and both of the parameters can be adjusted.

In [2]:
df = pd.read_csv('data/train.csv')
listing_id = 1029310
k = 10

In [3]:
# target product
df[df['listing_id'] == listing_id][['listing_id', 'title', 'category','description', 'power', 'model', 'engine_cap', 'mileage', 'price']]

,listing_id,title,category,description,power,model,engine_cap,mileage,price
14667,1029310,Kia Cerato K3 1.6A,"parf car, premium ad car, low mileage car",100% quality buy. low mileage and accident fre...,95.3,cerato,1591.0,31000.0,72400.0


### Step 1:  Recommendations  on Keywords

* Combine ‘manufacturer’, ‘model’ and ‘category’ together as keywords.
* Convert the combination into a word frequency matrix.
* Use distance to measure the similarity between products.
* Filter out top-200 similar products.

In [4]:
def get_top_recommendations_step_1(df, listing_id):
    
    # Reformat 'category'
    def get_category(cat):
        cat_list = cat.split(',')
        cat_list_reformat = [c.replace(' ','').replace('car','') for c in cat_list]
        return ' '.join(cat_list_reformat)
    
    
    # Move the selected car to the first line
    target_row = df[df['listing_id'] == listing_id]
    rows = df[df['listing_id'] != listing_id]
    df = pd.concat([target_row, rows]).reset_index(drop = True)
    
    # Create a new row to solve the null values in 'make'
    df['manufacturer'] = df['title'].str.split().str[0]
    # Choose key features
    features = ['listing_id', 'manufacturer', 'model', 'category']
    df_step_1 = df[features]
    df_step_1 = df_step_1.dropna()
    
    # Convert the collection of text to a matrix of token counts
    df_step_1.category = df_step_1.category.apply(get_category)
    df_step_1['keywords'] = df_step_1['manufacturer'] + ' ' + df_step_1['model'] + ' ' +  df_step_1['category']
    count = CountVectorizer(lowercase = True)
    count_matrix = count.fit_transform(df_step_1['keywords'])
    
    # Calculate the similarity
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # As the similarity of itself will be 1 and be placed in the front, the index starts from 1.
    sim_scores_200 = sim_scores_sorted[1:201]
    
    car_indices = [i[0] for i in sim_scores_200]

    return pd.DataFrame(df_step_1['listing_id'].iloc[car_indices])

In [5]:
# Results of Step 1
df_step_1 = get_top_recommendations_step_1(df, listing_id)
recommendation_1 = pd.merge(df_step_1, df)
recommendation_1['manufacturer'] = recommendation_1['title'].str.split().str[0]
recommendation_1[['listing_id', 'manufacturer', 'model', 'category', 'price']]

,listing_id,manufacturer,model,category,price
0,1010001,Kia,cerato,"parf car, premium ad car, low mileage car",62500.0
1,1020378,Kia,cerato,"parf car, premium ad car, low mileage car",84500.0
2,992578,Kia,cerato,"parf car, premium ad car, low mileage car",79000.0
3,1017422,Kia,cerato,"parf car, premium ad car, low mileage car",70700.0
4,1027987,Kia,cerato,"parf car, premium ad car, low mileage car",68000.0
...,...,...,...,...,...
195,980389,Kia,cerato,parf car,64700.0
196,1008536,Kia,cerato,parf car,83500.0
197,1018507,Kia,cerato,parf car,65800.0
198,971957,Kia,cerato,parf car,85600.0


### Step 2:  Recommendations on Price

* Base on the recommendation results of step 1 
* 80 recommendations with similar prices to the known car
* 10 highest-priced recommendations and the 10 lowest-priced recommendations

In [6]:
def get_top_recommendations_step_2(df, df_step_1, listing_id):
    
    # Start based on the result of Recommender 1
    top_200 = pd.merge(df_step_1, df)
    features = ['listing_id', 'price']
    df_step_2 = top_200[features]
    
    # Find the 10 highest and 10 lowest priced similar products
    price_sorted = df_step_2.sort_values(by='price', ascending=True)
    price_highest_10 = price_sorted[-10:]
    price_lowest_10 = price_sorted[:10]
    
    # Find products with similar prices
    target_price = df[df['listing_id'] == listing_id]['price']
    df_step_2['diff_price'] = df_step_2['price'].apply(lambda x: np.abs(x - target_price))
    diff_price_sorted = df_step_2.sort_values(by='diff_price', ascending=True)
    price_nearest_80 = diff_price_sorted[:80]
    
    df_price = pd.concat([price_nearest_80, price_lowest_10])
    df_price = pd.concat([df_price, price_highest_10])
    df_price = df_price.drop_duplicates()
    
    return pd.DataFrame(df_price['listing_id'])

In [7]:
# Results of Step 2
df_step_2 = get_top_recommendations_step_2(df, df_step_1, listing_id)
recommendations2 =  pd.merge(df_step_2, df)
target_price = df[df['listing_id'] == listing_id]['price']
recommendations2['diff_price'] = recommendations2['price'].apply(lambda x: x - target_price)
recommendations2[['listing_id', 'title', 'category', 'price', 'diff_price']]

,listing_id,title,category,price,diff_price
0,1012565,Kia Cerato K3 1.6A SX Sunroof,"parf car, premium ad car",72400.0,0.0
1,1024380,Kia Cerato 1.6A LX,"parf car, premium ad car",72400.0,0.0
2,1002653,Kia Cerato Forte 1.6A SX (New 10-yr COE),"coe car, premium ad car, low mileage car",72400.0,0.0
3,1015736,Kia Cerato K3 1.6A,"parf car, low mileage car",72400.0,0.0
4,1027631,Kia Cerato K3 1.6A Sunroof,"parf car, premium ad car",72400.0,0.0
...,...,...,...,...,...
95,1022159,Kia Cerato 1.6A GT Line Sunroof,"parf car, almost new car, premium ad car",99900.0,27500.0
96,1016231,Kia Cerato 1.6A GT Line Sunroof,"parf car, almost new car, premium ad car",102200.0,29800.0
97,1023538,Kia Cerato 1.6A GT Line Sunroof,"parf car, almost new car, premium ad car, low ...",105500.0,33100.0
98,1006086,Kia Stinger 3.3A GT Sunroof,"parf car, premium ad car, low mileage car",182400.0,110000.0


### Step 3: Recommendation on Content

* Based on the recommendation results of Step 2
* Calculate term frequency-inverse document frequency (TF-IDF) on the free text attribute `description`.
* Compute linear kernel between the target product to find similar items.
* Filter out top-k similar products.

In [8]:
def get_top_recommendations_step_3(df, df_step_2, listing_id, k):
    
    # Filter out invalid description
    def filter_description(line):
        try:
            line =  float(line)
            return None
        except:
            return line
    
    
    # Start based on the result of Recommender 2
    target_row = pd.DataFrame({'listing_id': listing_id, 'description': df[df.listing_id == listing_id]['description']})
    top_100 = pd.merge(df_step_2, df)
    top_100_plus = pd.concat([target_row, top_100]).reset_index(drop = True)
    features = ['listing_id', 'description']
    df_step_3 = top_100[features]
    df_step_3.description = df_step_3.description.apply(filter_description)
    df_step_3 = df_step_3.dropna()
    
    
    # Convert a collection of content to a matrix of TF-IDF features.
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df_step_3['description'])
    
    # Calculate the similarity
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores_sorted = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores_k = sim_scores[1:k+1]

    car_indices = [i[0] for i in sim_scores_k]

    return pd.DataFrame(df_step_3['listing_id'].iloc[car_indices])

In [9]:
df_step_3 =  get_top_recommendations_step_3(df, df_step_2, listing_id, k)
recommendations3 =  pd.merge(df_step_3, df)
target_price = df[df['listing_id'] == listing_id]['price']
recommendations3['diff_price'] = recommendations3['price'].apply(lambda x: x - target_price)
recommendations3[['listing_id', 'title', 'category', 'description', 'price', 'diff_price']]

,listing_id,title,category,description,price,diff_price
0,1024380,Kia Cerato 1.6A LX,"parf car, premium ad car",c&c 5 years warranty. 10 years engine warranty...,72400.0,0.0
1,1002653,Kia Cerato Forte 1.6A SX (New 10-yr COE),"coe car, premium ad car, low mileage car",3 years warranty coverage for engine & gearbox...,72400.0,0.0
2,1015736,Kia Cerato K3 1.6A,"parf car, low mileage car",1 owner only! low mileage clocked! agent servi...,72400.0,0.0
3,1027631,Kia Cerato K3 1.6A Sunroof,"parf car, premium ad car",cycle & carriage unit. one owner. metallic gre...,72400.0,0.0
4,1019001,Kia Cerato 1.6A EX (OPC),"opc car, parf car, low mileage car",1 owner only. agent unit. 100% loan available....,71300.0,-1100.0
5,1023314,Kia Cerato 1.6A EX,"parf car, premium ad car",lowest depreciation. agent serviced and warran...,73500.0,1100.0
6,1022238,Kia Cerato K3 1.6A Sunroof,parf car,"c&c unit, 5-year unlimited mileage warranty, 1...",70200.0,-2200.0
7,1019354,Kia Cerato K3 1.6A,"parf car, low mileage car",one of the most popular sedans! excellent fuel...,70200.0,-2200.0
8,1012795,Kia Cerato K3 1.6A,"parf car, premium ad car","signature 1 owner k3, valid agent warranty, ag...",74600.0,2200.0
9,1015066,Kia Cerato K3 1.6A SX Sunroof,"parf car, low mileage car","100% guaranteed accident free, 1 owner, sunroo...",70200.0,-2200.0


## Final Recommendation Engine

In [10]:
def get_top_recommendations(df, listing_id, k):
   
    df_step_1 = get_top_recommendations_step_1(df, listing_id) 
    df_step_2 = get_top_recommendations_step_2(df, df_step_1, listing_id)
    df_step_3 = get_top_recommendations_step_3(df, df_step_2, listing_id, k)
    
    top_k = pd.merge(df_step_3, df)
    target_price = df[df['listing_id'] == listing_id]['price']
    top_k['diff_price'] = top_k['price'].apply(lambda x: x - target_price)
    
    return top_k

In [11]:
# Test the recommendation engine

# df = pd.read_csv('data/train.csv')
# listing_id = 1029310
# k = 10

recommendations =  get_top_recommendations(df, listing_id, k)
target_features = ['listing_id', 'title', 'manufactured', 'power', 'engine_cap', 'mileage', 'coe', 'price']
recommendations[target_features]

,listing_id,title,manufactured,power,engine_cap,mileage,coe,price
0,1024380,Kia Cerato 1.6A LX,2018.0,93.8,1591.0,51000.0,31997.0,72400.0
1,1002653,Kia Cerato Forte 1.6A SX (New 10-yr COE),2012.0,91.1,1591.0,91189.0,NaN,72400.0
2,1015736,Kia Cerato K3 1.6A,2017.0,95.3,1591.0,32000.0,38510.0,72400.0
3,1027631,Kia Cerato K3 1.6A Sunroof,2017.0,95.3,1591.0,49920.0,38830.0,72400.0
4,1019001,Kia Cerato 1.6A EX (OPC),2019.0,93.8,1591.0,16000.0,13000.0,71300.0
5,1023314,Kia Cerato 1.6A EX,2018.0,93.8,1591.0,NaN,33798.0,73500.0
6,1022238,Kia Cerato K3 1.6A Sunroof,2017.0,95.3,1591.0,52160.0,36001.0,70200.0
7,1019354,Kia Cerato K3 1.6A,2017.0,95.3,1591.0,29800.0,41617.0,70200.0
8,1012795,Kia Cerato K3 1.6A,2017.0,95.3,1591.0,42100.0,38510.0,74600.0
9,1015066,Kia Cerato K3 1.6A SX Sunroof,2017.0,95.3,1591.0,29856.0,46791.0,70200.0
